### initial analysis for species confirmation and identification of bad samples

In [ ]:
#input file
vcffile="prelim.recode.vcf"

#"databases"
samplefile="/home/megan/megan/research/eucalyptus/eucalyptus_data/Emelliodora_PlantsSamples.csv"
accessionfile="/home/megan/megan/research/eucalyptus/eucalyptus_data/Emelliodora_Accessions.csv"

#count header lines in command line (grep -c "##" batch_1.vcf)
skipr=8
#non sample columns
skipc=9

thresh.missing.loci=90  #higher number allows more missing data/keeps more loci
thresh.missing.samples=90  #higher number allows more missing data/keeps more samples

In [ ]:
library(oz)
options(jupyter.plot_mimetypes = "image/png")

In [ ]:
#read in vcf, skupping ##header rows
vcf=read.delim(vcffile, sep="\t", header=T, skip=skipr)
#remove non sample columns
vcf=vcf[c(-skipc:-1)]
#get just genotypes
genos=apply(vcf,2,substr,1,3)
#transpose 
genos=t(genos)
#clean sample names
row.names(genos)=substr(row.names(genos),9,14)
#genos[1:5,1:10]

In [ ]:
#add sample metadata
sampleinfo=read.csv(samplefile, header=T)  #read in sample information
samplematches=match(sampleinfo$SampleID, row.names(genos))  #subset for just samples in this analysis
samples=sampleinfo[!is.na(samplematches),][order(na.omit(samplematches)),]

#incorporate latitude and longitude from accession
accessioninfo=read.csv(accessionfile, header=T)  #read in accession information
accessionmatches=match(samples$AccessionID, accessioninfo$AccessionID)
samples=cbind(samples,accessioninfo$Latitude[accessionmatches],accessioninfo$Longitude[accessionmatches],
              accessioninfo$PopulationName[accessionmatches])
names(samples)[names(samples)=="accessioninfo$Latitude[accessionmatches]"]="Latitude"
names(samples)[names(samples)=="accessioninfo$Longitude[accessionmatches]"]="Longitude"
names(samples)[names(samples)=="accessioninfo$PopulationName[accessionmatches]"]="PopulationName"

In [ ]:
#map samples
oz(ylim=c(min(samples$Latitude)-1,max(samples$Latitude)+1),
   xlim=c(min(samples$Longitude)-1, max(samples$Longitude)+1))
text(samples$Longitude,samples$Latitude, 
     label=samples$PopulationName, cex = .6) #add population names
abline(v=130) #use to define boundary for geographic outliers

In [ ]:
#identify geographic outliers
minlong=130  #set minimimum longitude for filtering out geographic outliers
geog.out=as.character(samples[which(samples$Longitude < minlong),]$SampleID)
length(geog.out)
geog.out

In [ ]:
dim(genos)

#identify missing data
missing.filt=genos=="./."                 

#identify bad loci
missPerLocus.filt=colSums(missing.filt)   #count missing data per locus
locus.filt=which(missPerLocus.filt < thresh.missing.loci / 100 * nrow(genos))

#identify bad samples
missPerSample.filt=rowSums(missing.filt)
ind.filt=which(missPerSample.filt < thresh.missing.samples / 100 * ncol(genos))

#filter
genos.filt=genos[ind.filt,locus.filt]
dim(genos.filt)
failed=names(which(missPerSample.filt > thresh.missing.samples / 100 * ncol(genos)))
length(failed)
failed

In [ ]:
#put in matrix format
genos.filt[genos.filt == "./."] <- "NA"
genos.filt[genos.filt == "1/1"] <- "0"
genos.filt[genos.filt == "0/1"] <- "1"
genos.filt[genos.filt == "1/0"] <- "1"
genos.filt[genos.filt == "0/0"] <- "2"

In [ ]:
#calculate PCA
geno.dist=dist(genos.filt)  #generate distance matrix
sum(is.na(geno.dist))  #count NAs in it
pca=cmdscale(geno.dist,20, eig=T) #PCA

In [ ]:
#plot PCA
#pdf(file="pca.pdf")
pca=cmdscale(geno.dist,20,eig=T)
per_expl=round(pca$eig/sum(pca$eig)*100,digits=1)
plot(pca$points[,1:2], pch=".", 
    xlab=paste("PCA axis 1 (",per_expl[1],"%)"),
    ylab=paste("PCA axis 2 (",per_expl[2],"%)"))
text(pca$points[,1:2], label=rownames(pca$points), cex = .6)
abline(h=-30,v=-30)
#dev.off()

In [ ]:
#identify outlier species
minpc1=-30  #remove samples below this value on pc1
minpc2=-30  #remove samples below this value on pc2
sp.out=names(which(pca$points[,1]<minpc1 | pca$points[,2]<minpc2))
length(sp.out)
sp.out

In [ ]:
#filter to remove failed, geographic outliers, and species outliers
remove_ids=sort(unique(c(failed,geog.out,sp.out)))
length(remove_ids)
remove_ids

#remove samples
samples=samples[!samples$SampleID %in% remove_ids,]
dim(samples)[1]

In [ ]:
#map remaining samples
#pdf(file="map.pdf")
oz(ylim=c(min(samples$Latitude)-1,max(samples$Latitude)+1),
   xlim=c(min(samples$Longitude)-1, max(samples$Longitude)+1))
text(samples$Longitude,samples$Latitude, 
     label=samples$PopulationName, cex = .6) #add population names
#dev.off()

In [ ]:
##plot other PCA dimentions
#par(mfrow=c(3,3), mar=c(4,4,2,1))
#plot(pca$points[,c(1,2)], pch="")
#text(pca$points[,c(1,2)],label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(1,3)], pch="")
#text(pca$points[,c(1,3)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(1,4)], pch="")
#text(pca$points[,c(1,4)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(2,1)], pch="")
#text(pca$points[,c(2,1)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(2,3)], pch="")
#text(pca$points[,c(2,3)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(2,4)], pch="")
#text(pca$points[,c(2,4)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(3,1)], pch="")
#text(pca$points[,c(3,1)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(3,2)], pch="")
#text(pca$points[,c(3,2)], label=rownames(pca$points), cex = .6)
#plot(pca$points[,c(3,4)], pch="")
#text(pca$points[,c(3,4)], label=rownames(pca$points), cex = .6)

##plot dendrogram
#hc <- hclust(geno.dist)
#plot(hc, cex=.5)

##determine outliers
#assigns=cutree(hc, k=3)
#outliers2=subset(assigns, assigns==2)
#outliers3=subset(assigns, assigns==3)
#sp=subset(assigns, assigns==1)
#length(outliers2)
#outliers2
#length(outliers3)
#outliers3
#length(sp)
#assigns